# 🦠 Exploration Interactive - Articles COVID-19

Ce notebook permet d'explorer les résultats du projet de classification d'articles scientifiques.

**Projet:** Classification et analyse d'articles CORD-19 avec NLP et Graph Neural Networks

## 📦 1. Imports et Configuration

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

print("✅ Configuration terminée")

✅ Configuration terminée


In [4]:
pip install pandas numpy networkx matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import AutoTokenizer, AutoModel
import community as community_louvain
import umap

print("✅ Numpy:", np.__version__)
print("✅ Pandas:", pd.__version__)
print("✅ NetworkX:", nx.__version__)
print("✅ PyTorch:", torch.__version__)
print("✅ Transformers: OK")
print("✅ UMAP: OK")
print("✅ Community (Louvain): OK")
print("\n🎉 Tout fonctionne parfaitement !")

✅ Numpy: 2.3.4
✅ Pandas: 2.3.3
✅ NetworkX: 3.5
✅ PyTorch: 2.9.0+cpu
✅ Transformers: OK
✅ UMAP: OK
✅ Community (Louvain): OK

🎉 Tout fonctionne parfaitement !


In [9]:
pip install torch transformers python-louvain umap-learn


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached python-louvain-0.16.tar.gz (204 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached umap_learn-0.5.9.post2-py3-none-any.whl.metadata (25 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)
Using cached umap_learn-0.5.9.post2-

  DEPRECATION: Building 'python-louvain' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'python-louvain'. Discussion can be found at https://github.com/pypa/pip/issues/6334


## 📂 2. Chargement des Données

In [13]:
# Chemins
PROJECT_ROOT = Path(r"C:\Users\lapte\Desktop\AL\project\S1_CORD19_Classification")
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

print("📂 Chargement des données...")

# DataFrame
df = pd.read_csv(DATA_PROCESSED / "articles_with_communities.csv")
print(f"✅ {len(df):,} articles chargés")

# Embeddings
embeddings = np.load(DATA_PROCESSED / "embeddings.npy")
print(f"✅ Embeddings: {embeddings.shape}")

# Graphe
graph = nx.read_gpickle(DATA_PROCESSED / "article_graph.gpickle")
print(f"✅ Graphe: {graph.number_of_nodes():,} nœuds, {graph.number_of_edges():,} arêtes")

# Communautés
with open(DATA_PROCESSED / "article_graph_communities.pkl", 'rb') as f:
    communities = pickle.load(f)
print(f"✅ {len(set(communities.values()))} communautés")

print("\n✅ Toutes les données sont chargées!")

📂 Chargement des données...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lapte\\Desktop\\AL\\project\\S1_CORD19_Classification\\data\\processed\\articles_with_communities.csv'

## 📊 3. Vue d'Ensemble

In [ ]:
# Afficher les premières lignes
df.head()

In [ ]:
# Statistiques globales
print("📊 STATISTIQUES GLOBALES")
print("="*60)

print(f"\n📚 Dataset:")
print(f"   - Total articles: {len(df):,}")
print(f"   - Avec abstract: {df['abstract'].notna().sum():,}")
print(f"   - Longueur moyenne: {df['full_text'].str.len().mean():.0f} caractères")

print(f"\n🕸️  Graphe:")
print(f"   - Nœuds: {graph.number_of_nodes():,}")
print(f"   - Arêtes: {graph.number_of_edges():,}")
print(f"   - Densité: {nx.density(graph):.6f}")
print(f"   - Degré moyen: {sum(dict(graph.degree()).values()) / graph.number_of_nodes():.2f}")

comm_sizes = pd.Series(communities.values()).value_counts()
print(f"\n🎯 Communautés:")
print(f"   - Nombre: {len(set(communities.values()))}")
print(f"   - Taille moyenne: {comm_sizes.mean():.1f} articles")
print(f"   - Plus grande: {comm_sizes.max()} articles")

## 📈 4. Distribution des Communautés

In [ ]:
comm_sizes = pd.Series(communities.values()).value_counts().sort_values(ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Distribution complète
ax1.bar(range(len(comm_sizes)), comm_sizes.values, color='steelblue', alpha=0.7)
ax1.set_xlabel('Communauté (classée par taille)', fontsize=12)
ax1.set_ylabel('Nombre d\'articles', fontsize=12)
ax1.set_title('Distribution des Tailles de Communautés', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Top 10
top10 = comm_sizes.head(10)
ax2.barh(range(len(top10)), top10.values, color='coral', alpha=0.7)
ax2.set_yticks(range(len(top10)))
ax2.set_yticklabels([f"Communauté {idx}" for idx in top10.index])
ax2.set_xlabel('Nombre d\'articles', fontsize=12)
ax2.set_title('Top 10 Plus Grandes Communautés', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n📊 Top 10 communautés:")
for i, (comm_id, size) in enumerate(top10.items(), 1):
    print(f"   {i}. Communauté {comm_id}: {size} articles")

## 🔍 5. Explorer une Communauté

**Modifiez la variable `COMMUNITY_ID` ci-dessous pour explorer différentes communautés**

In [ ]:
# ⚙️ PARAMÈTRE À MODIFIER
COMMUNITY_ID = 0  # Changer ce numéro

print(f"🔍 EXPLORATION DE LA COMMUNAUTÉ {COMMUNITY_ID}")
print("="*60)

# Filtrer les articles
comm_articles = df[df.index.map(lambda x: communities.get(x, -1) == COMMUNITY_ID)]

print(f"\n📚 Nombre d'articles: {len(comm_articles)}")

# Sous-graphe
subgraph = graph.subgraph(comm_articles.index.tolist())
print(f"\n🕸️  Graphe de la communauté:")
print(f"   - Nœuds: {subgraph.number_of_nodes()}")
print(f"   - Arêtes: {subgraph.number_of_edges()}")
print(f"   - Densité: {nx.density(subgraph):.4f}")

# Articles centraux
if len(subgraph) > 0:
    centrality = nx.degree_centrality(subgraph)
    top_central = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print(f"\n⭐ Top 5 articles les plus centraux:")
    for i, (idx, cent) in enumerate(top_central, 1):
        print(f"   {i}. [{cent:.3f}] {df.iloc[idx]['title'][:70]}...")

# Exemples de titres
print(f"\n📄 Exemples de titres:")
for i, title in enumerate(comm_articles['title'].head(10), 1):
    print(f"   {i}. {title[:80]}...")

## 🎨 6. Visualiser la Communauté

In [ ]:
def visualize_community(community_id, max_nodes=100):
    """Visualise une communauté"""
    comm_articles = df[df.index.map(lambda x: communities.get(x) == community_id)]
    subgraph = graph.subgraph(comm_articles.index.tolist())
    
    # Limiter si trop grand
    if len(subgraph) > max_nodes:
        degrees = dict(subgraph.degree())
        top_nodes = sorted(degrees, key=degrees.get, reverse=True)[:max_nodes]
        subgraph = subgraph.subgraph(top_nodes)
        print(f"⚠️  Affichage limité aux {max_nodes} nœuds les plus connectés")
    
    # Layout
    pos = nx.spring_layout(subgraph, k=0.5, iterations=50, seed=42)
    
    # Tailles basées sur le degré
    degrees = dict(subgraph.degree())
    node_sizes = [degrees[node] * 50 + 50 for node in subgraph.nodes()]
    
    # Visualisation
    plt.figure(figsize=(16, 12))
    
    nx.draw_networkx_nodes(subgraph, pos, node_size=node_sizes,
                          node_color='lightblue', alpha=0.7, 
                          edgecolors='black', linewidths=1)
    nx.draw_networkx_edges(subgraph, pos, alpha=0.3, width=1)
    
    # Labels pour les 5 plus centraux
    centrality = nx.degree_centrality(subgraph)
    top_5 = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    labels = {node: df.iloc[node]['title'][:25] + '...' for node, _ in top_5}
    nx.draw_networkx_labels(subgraph, pos, labels, font_size=9, font_weight='bold')
    
    plt.title(f"Communauté {community_id}\n{len(comm_articles)} articles, "
             f"{subgraph.number_of_edges()} connexions",
             fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualiser
visualize_community(COMMUNITY_ID, max_nodes=100)

## 🔎 7. Recherche Sémantique

**Modifiez la variable `QUERY` pour rechercher différents sujets**

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

def search_articles(query, top_k=10):
    """Recherche sémantique"""
    # Charger le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"💻 Device: {device}")
    
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)
    model.eval()
    
    print(f"🔍 Recherche: '{query}'...")
    
    # Encoder la requête
    inputs = tokenizer(query, padding=True, truncation=True, 
                      max_length=512, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        query_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()[0]
    
    # Normaliser
    query_emb = query_emb / np.linalg.norm(query_emb)
    norm_emb = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    # Similarités
    similarities = np.dot(norm_emb, query_emb)
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # Résultats
    results = df.iloc[top_indices].copy()
    results['similarity_score'] = similarities[top_indices]
    results['community'] = results.index.map(communities)
    
    return results[['title', 'abstract', 'similarity_score', 'community']]

In [ ]:
# ⚙️ PARAMÈTRE À MODIFIER
QUERY = "COVID-19 vaccine effectiveness"  # Modifier votre requête ici

print("\n" + "="*60)
print("RECHERCHE SÉMANTIQUE")
print("="*60)

results = search_articles(QUERY, top_k=10)

print(f"\n📚 Résultats pour: '{QUERY}'\n")
for i, (idx, row) in enumerate(results.iterrows(), 1):
    print(f"{i}. [Score: {row['similarity_score']:.3f}] [Comm: {row['community']}]")
    print(f"   {row['title']}")
    if pd.notna(row['abstract']):
        print(f"   {row['abstract'][:150]}...")
    print()

## 🔗 8. Articles Similaires

**Trouvez des articles similaires à un article donné**

In [ ]:
# ⚙️ PARAMÈTRE À MODIFIER
ARTICLE_INDEX = 0  # Changer cet index

print("\n" + "="*60)
print("ARTICLES SIMILAIRES")
print("="*60)

print(f"\n📄 Article de référence (index {ARTICLE_INDEX}):")
print(f"   {df.iloc[ARTICLE_INDEX]['title']}")
print(f"   Communauté: {communities.get(ARTICLE_INDEX)}\n")

# Calculer similarités
norm_emb = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
ref_emb = norm_emb[ARTICLE_INDEX]
similarities = np.dot(norm_emb, ref_emb)
similarities[ARTICLE_INDEX] = -1

top_indices = np.argsort(similarities)[::-1][:10]
similar = df.iloc[top_indices].copy()
similar['similarity_score'] = similarities[top_indices]
similar['community'] = similar.index.map(communities)

print(f"🔗 Top 10 articles similaires:\n")
for i, (idx, row) in enumerate(similar.iterrows(), 1):
    print(f"{i}. [Score: {row['similarity_score']:.3f}] [Comm: {row['community']}]")
    print(f"   {row['title']}\n")

## 📊 9. Analyse des Mots-Clés

In [ ]:
def analyze_keywords(community_id, top_n=20):
    """Analyse les mots-clés d'une communauté"""
    comm_articles = df[df.index.map(lambda x: communities.get(x) == community_id)]
    all_text = ' '.join(comm_articles['title'].dropna().tolist()).lower()
    
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 
                'for', 'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are'}
    
    words = all_text.split()
    word_counts = {}
    for word in words:
        word = word.strip('.,;:!?"()[]{}') 
        if len(word) > 3 and word not in stopwords and not word.isdigit():
            word_counts[word] = word_counts.get(word, 0) + 1
    
    return sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Analyser les top communautés
print("📊 MOTS-CLÉS PAR COMMUNAUTÉ")
print("="*60)

top_comms = comm_sizes.head(5)
for comm_id in top_comms.index:
    keywords = analyze_keywords(comm_id, top_n=10)
    print(f"\n🎯 Communauté {comm_id} ({top_comms[comm_id]} articles):")
    print("   Top 10 mots-clés:")
    for i, (word, count) in enumerate(keywords, 1):
        print(f"      {i}. {word} ({count})")

## 💾 10. Export des Résultats

In [ ]:
# Exporter une communauté spécifique
COMM_TO_EXPORT = 0

OUTPUT_DIR = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

comm_articles = df[df.index.map(lambda x: communities.get(x) == COMM_TO_EXPORT)]
export_path = OUTPUT_DIR / f"community_{COMM_TO_EXPORT}_articles.csv"
comm_articles.to_csv(export_path, index=False)

print(f"✅ Communauté {COMM_TO_EXPORT} exportée: {export_path}")
print(f"   {len(comm_articles)} articles sauvegardés")

## ✅ Fin de l'Exploration

Vous pouvez maintenant:
- Modifier les paramètres (COMMUNITY_ID, QUERY, ARTICLE_INDEX)
- Ré-exécuter les cellules pour explorer d'autres parties du dataset
- Créer vos propres analyses personnalisées